In [33]:
# Data visualisation practice: See CO trace
# 2017-09-20
#Filip
#Kevin

import plotly.offline as py
import pandas as pd
import requests

df_airports = pd.read_excel(r'location.xlsx')
df_flight_paths = pd.read_excel(r'PU&WF&CO.xlsx')

,iata,airport,lat,long
0,NaN,SDC Gdansk,3.364044e+09,-8.442694e+09
1,NaN,SDC Bangalore,3.289595e+09,-9.703720e+05
2,NaN,SDC Gdansk,3.343417e+09,-1.120081e+09
3,NaN,SDC Bangalore,3.985841e+09,-1.046670e+09
4,NaN,SDC Gdansk,2.998047e+09,-9.533972e+09


In [114]:
df_flight_paths[['GRID ID','WF','CO location']].isnull().any()
df_flight_paths.dropna(inplace=True)

df_flight_paths.CDU=df_flight_paths.CDU.apply(lambda x:x.replace('SDC ',''))
df_flight_paths.CDU=df_flight_paths.CDU.apply(lambda x:x.replace('CDU ',''))
df_flight_paths.CDU=df_flight_paths.CDU.apply(lambda x:x.replace('Local ',''))

df_flight_paths['CO location']=df_flight_paths['CO location'].apply(lambda x:x.replace('FI Global KYC CO','Amsterdam'))
df_flight_paths['CO location']=df_flight_paths['CO location'].apply(lambda x:x.replace('FM Global KYC CO','Eindhoven'))
df_flight_paths['CO location']=df_flight_paths['CO location'].apply(lambda x:x.replace('KYC CO (KZ)',''))
df_flight_paths['CO location']=df_flight_paths['CO location'].apply(lambda x:x.replace('KYC CO (Nordic)',''))
df_flight_paths['CO location']=df_flight_paths['CO location'].apply(lambda x:x.replace('KYC CO',''))
df_flight_paths['CO location']=df_flight_paths['CO location'].apply(lambda x:x.replace('TBD','Beijing'))
df_flight_paths['CO location']=df_flight_paths['CO location'].apply(lambda x:x.replace('Trust','Amsterdam'))
df_flight_paths['CO location']=df_flight_paths['CO location'].apply(lambda x:x.replace(' ',''))

df_flight_paths['WF_NUMBER']=df_flight_paths.WF.apply(lambda x: int(x.split(".")[0]) if x.split(".")[0]!='Old status' else 0)
df_flight_paths['Color_Category']=df_flight_paths.WF_NUMBER.apply(lambda x: 'grey' \
                                                                  if x< 18 else 'red' \
                                                                  if x in [18,19,20,25,26] else 'yellow' \
                                                                  if x in [21,22,23,27,28,29,30] else 'green')
df_flight_paths.drop('WF_NUMBER',axis=1,inplace=True)

In [34]:
# Check is there any null in locations
df_airports=df_airports[["airport",'lat','long']]
df_airports.airport.isnull().any()
# Only keep country and location name
df_airports.airport=df_airports.airport.apply(lambda x:x.replace('SDC ',''))
df_airports.airport=df_airports.airport.apply(lambda x:x.replace('CDU ',''))
df_airports.airport=df_airports.airport.apply(lambda x:x.replace('Local ',''))
df_airports.airport=df_airports.airport.apply(lambda x:x.replace('FI Global KYC CO','Amsterdam'))
df_airports.airport=df_airports.airport.apply(lambda x:x.replace('FM Global KYC CO','Eindhoven'))
df_airports.airport=df_airports.airport.apply(lambda x:x.replace('KYC CO (KZ)',''))
df_airports.airport=df_airports.airport.apply(lambda x:x.replace('KYC CO (Nordic)',''))
df_airports.airport=df_airports.airport.apply(lambda x:x.replace('KYC CO',''))
df_airports.airport=df_airports.airport.apply(lambda x:x.replace('TBD','Beijing'))
df_airports.airport=df_airports.airport.apply(lambda x:x.replace('Trust','Amsterdam'))
df_airports.airport=df_airports.airport.apply(lambda x:x.replace(' ',''))

False

In [59]:
# get unique locations
location_list=df_airports.airport.unique().tolist()

# get GPS locations
location_list=df_airports.airport.unique()
GeoData={}
api_key = ""
for x in location_list:
    try:
        country=x
        response = requests.get('https://maps.googleapis.com/maps/api/geocode/json?address={0}&key={1}'.format(x, api_key))
        resp_json_payload = response.json()
        print(resp_json_payload['results'][0]['geometry']['location'])
        temp=resp_json_payload['results'][0]['geometry']['location']
        GeoData[country]=temp
    except:
        pass

Locations_GPS=pd.DataFrame(GeoData).T.reset_index()

{'lat': 54.35202520000001, 'lng': 18.6466384}
{'lat': 12.9715987, 'lng': 77.5945627}
{'lat': 14.5995124, 'lng': 120.9842195}
{'lat': 51.5073509, 'lng': -0.1277583}
{'lat': 49.815273, 'lng': 6.129582999999999}
{'lat': 48.1485965, 'lng': 17.1077478}
{'lat': 51.919438, 'lng': 19.145136}
{'lat': 46.818188, 'lng': 8.227511999999999}
{'lat': 52.3702157, 'lng': 4.895167900000001}
{'lat': 40.7127837, 'lng': -74.0059413}
{'lat': 45.943161, 'lng': 24.96676}
{'lat': 38.963745, 'lng': 35.243322}
{'lat': 51.165691, 'lng': 10.451526}
{'lat': 55.378051, 'lng': -3.435973}
{'lat': 46.227638, 'lng': 2.213749}
{'lat': 50.503887, 'lng': 4.469936}
{'lat': 52.132633, 'lng': 5.291265999999999}
{'lat': 1.352083, 'lng': 103.819836}
{'lat': 51.44164199999999, 'lng': 5.4697225}
{'lat': 36.204824, 'lng': 138.252924}
{'lat': 54.5259614, 'lng': -105.2551187}
{'lat': 23.69781, 'lng': 120.960515}
{'lat': 48.379433, 'lng': 31.1655799}
{'lat': 41.87194, 'lng': 12.56738}
{'lat': 35.86166, 'lng': 104.195397}
{'lat': 35.9

In [75]:
df_location_filled=df_airports[['airport']].merge(Locations_GPS,left_on='airport',right_on='index',how='left')
df_location_filled.drop('index', axis=1, inplace=True)
df_location_filled.rename(columns={'lng':'long'},inplace=True)

In [115]:
df_flight_paths_1=df_flight_paths.merge(Locations_GPS,left_on='CDU',right_on='index',how='left')
df_flight_paths_filled=df_flight_paths_1.merge(Locations_GPS,left_on='CO location',right_on='index',how='left',suffixes=('_start', '_end'))
df_flight_paths_filled.drop(['index_start','index_end',],axis=1,inplace=True)

In [130]:
# The one below is "df_airport",'df_flight_paths'
df_location_filled.to_excel(r'C:\Users\gzhang080\Documents\Mystaff\Project\KYC\GitTest\DataFly\df_location.xlsx')
df_flight_paths_filled.to_excel(r'C:\Users\gzhang080\Documents\Mystaff\Project\KYC\GitTest\DataFly\df_flight_paths.xlsx')

In [ ]:

airports = [ dict(
        type = 'scattergeo',
        locationmode = 'USA-states',
        lon = df_airports['long'],
        lat = df_airports['lat'],
        hoverinfo = 'text',
        text = df_airports['airport'],
        mode = 'markers',
        marker = dict( 
            size=2, 
            color='rgb(255, 0, 0)',
            line = dict(
                width=3,
                color='rgba(68, 68, 68, 0)'
            )
        ))]
        
flight_paths = []
for i in range( len( df_flight_paths ) ):
    flight_paths.append(
        dict(
            type = 'scattergeo',
            locationmode = 'USA-states',
            lon = [ df_flight_paths['start_lon'][i], df_flight_paths['end_lon'][i] ],
            lat = [ df_flight_paths['start_lat'][i], df_flight_paths['end_lat'][i] ],
            mode = 'lines',
            line = dict(
                width = 1,
                color = 'red',
            ),
            opacity = float(df_flight_paths['cnt'][i])/float(df_flight_paths['cnt'].max()),
        )
    )
    
layout = dict(
        title = 'Feb. 2011 American Airline flight paths<br>(Hover for airport names)',
        showlegend = False, 
        geo = dict(
            scope='China',
            projection=dict( type='azimuthal equal area' ),
            showland = True,
            landcolor = 'rgb(243, 243, 243)',
            countrycolor = 'rgb(204, 204, 204)',
        ),
    )

In [ ]:
flight_paths

In [ ]:
fig = dict( data=flight_paths + 
           airports, layout=layout )
py.plot( fig, filename='d3-flight-paths' )